# GPU-Accelerated Model Evaluation (50 Features)

This notebook evaluates the 3 LSTM models (BTC, ETH, SOL) with 50-feature datasets on GPU.

**Models**: `lstm_{SYMBOL}_USD_1m_7b5f0829.pt` (128/3/bidirectional, 1M+ params)

**Setup**:
1. Runtime → Change runtime type → GPU (T4)
2. Upload model files and feature datasets
3. Run all cells

In [ ]:
# Check GPU availability
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

In [ ]:
# Install dependencies
!pip install -q loguru pandas pyarrow scikit-learn numpy

In [ ]:
# Create directory structure
!mkdir -p models/new data/features scripts apps/trainer/{models,train,eval} libs/{constants,rl_env}

## Option A: Upload Files from Local Machine

Upload these files using the Colab file browser (left sidebar):

**Models** (3.9 MB each → `models/new/`):
- `lstm_BTC_USD_1m_7b5f0829.pt`
- `lstm_ETH_USD_1m_7b5f0829.pt`
- `lstm_SOL_USD_1m_7b5f0829.pt`

**Feature Datasets** (200+ MB each → `data/features/`):
- `features_BTC-USD_1m_2025-11-13_50feat.parquet`
- `features_ETH-USD_1m_2025-11-13_50feat.parquet`
- `features_SOL-USD_1m_2025-11-13_50feat.parquet`

## Option B: Download from Google Drive

If you've uploaded files to Google Drive, mount it and copy:

In [ ]:
# Uncomment to use Google Drive
# from google.colab import drive
# drive.mount('/content/drive')

# Copy files from Google Drive
# !cp /content/drive/MyDrive/crpbot/models/new/*.pt models/new/
# !cp /content/drive/MyDrive/crpbot/data/features/*_50feat.parquet data/features/

In [ ]:
# Verify files uploaded
!ls -lh models/new/
!ls -lh data/features/

## Setup Code Modules

We need to recreate the necessary Python modules for evaluation.

In [ ]:
%%writefile apps/trainer/models/lstm.py
"""LSTM model for direction prediction."""
import torch
import torch.nn as nn

class LSTMModel(nn.Module):
    """Bidirectional LSTM for binary classification."""

    def __init__(
        self,
        input_size: int,
        hidden_size: int = 128,
        num_layers: int = 3,
        dropout: float = 0.2,
        bidirectional: bool = True,
    ):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.bidirectional = bidirectional

        self.lstm = nn.LSTM(
            input_size,
            hidden_size,
            num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0,
            bidirectional=bidirectional,
        )

        lstm_output_size = hidden_size * 2 if bidirectional else hidden_size
        self.fc1 = nn.Linear(lstm_output_size, 64)
        self.dropout = nn.Dropout(dropout)
        self.fc2 = nn.Linear(64, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """Forward pass.
        
        Args:
            x: (batch_size, seq_len, input_size)
            
        Returns:
            Predictions: (batch_size, 1)
        """
        lstm_out, _ = self.lstm(x)  # (batch, seq, hidden*2)
        last_output = lstm_out[:, -1, :]  # (batch, hidden*2)
        x = torch.relu(self.fc1(last_output))
        x = self.dropout(x)
        x = self.fc2(x)
        return self.sigmoid(x)

In [ ]:
%%writefile apps/trainer/data_pipeline.py
"""Data loading and preprocessing."""
from pathlib import Path
from typing import Tuple

import numpy as np
import pandas as pd
from loguru import logger

def load_features_from_parquet(symbol: str, features_dir: str = "data/features") -> pd.DataFrame:
    """Load features from parquet."""
    features_path = Path(features_dir) / f"features_{symbol}_1m_latest.parquet"
    if not features_path.exists():
        # Try dated file
        dated_files = list(Path(features_dir).glob(f"features_{symbol}_1m_*_50feat.parquet"))
        if dated_files:
            features_path = sorted(dated_files)[-1]  # Use latest
        else:
            raise FileNotFoundError(f"No feature file found for {symbol}")
    
    df = pd.read_parquet(features_path)
    logger.info(f"Loaded features from {features_path}: {len(df):,} rows, {len(df.columns)} columns")
    return df

def create_walk_forward_splits(
    df: pd.DataFrame,
    train_ratio: float = 0.7,
    val_ratio: float = 0.15,
) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """Split data chronologically."""
    n = len(df)
    train_end = int(n * train_ratio)
    val_end = int(n * (train_ratio + val_ratio))
    
    train_df = df.iloc[:train_end].copy()
    val_df = df.iloc[train_end:val_end].copy()
    test_df = df.iloc[val_end:].copy()
    
    logger.info(f"Creating walk-forward splits: train until {train_df['timestamp'].max()}, val until {val_df['timestamp'].max()}")
    logger.info(f"Split sizes: Train={len(train_df)}, Val={len(val_df)}, Test={len(test_df)}")
    
    return train_df, val_df, test_df

In [ ]:
%%writefile apps/trainer/features.py
"""Feature normalization."""
from typing import List, Tuple
import numpy as np
import pandas as pd
from loguru import logger

FEATURE_COLUMNS = [
    # Session features (5)
    "session_tokyo", "session_london", "session_ny", "day_of_week", "is_weekend",
    # Spread features (4)
    "spread", "spread_pct", "atr", "spread_atr_ratio",
    # Volume features (3)
    "volume_ma", "volume_ratio", "volume_trend",
    # Moving averages (8)
    "sma_7", "sma_14", "sma_21", "sma_50",
    "price_to_sma_7", "price_to_sma_14", "price_to_sma_21", "price_to_sma_50",
    # Technical indicators (8)
    "rsi", "macd", "macd_signal", "macd_diff",
    "bb_upper", "bb_middle", "bb_lower", "bb_width",
    # Multi-TF OHLCV (15)
    "5m_open", "5m_high", "5m_low", "5m_close", "5m_volume",
    "15m_open", "15m_high", "15m_low", "15m_close", "15m_volume",
    "1h_open", "1h_high", "1h_low", "1h_close", "1h_volume",
    # Cross-TF alignment (3)
    "cross_tf_alignment_score", "cross_tf_alignment_direction", "cross_tf_alignment_strength",
    # Volatility (1)
    "atr_percentile",
]

def normalize_features(
    train_df: pd.DataFrame,
    val_df: pd.DataFrame = None,
    test_df: pd.DataFrame = None,
    method: str = "standard",
) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, dict]:
    """Normalize features using training set statistics."""
    feature_cols = [c for c in FEATURE_COLUMNS if c in train_df.columns]
    
    if method == "standard":
        stats = {
            "mean": train_df[feature_cols].mean().to_dict(),
            "std": train_df[feature_cols].std().to_dict(),
        }
        
        for col in feature_cols:
            mean, std = stats["mean"][col], stats["std"][col]
            if std > 0:
                train_df[col] = (train_df[col] - mean) / std
                if val_df is not None:
                    val_df[col] = (val_df[col] - mean) / std
                if test_df is not None:
                    test_df[col] = (test_df[col] - mean) / std
    
    logger.info(f"Normalized {len(feature_cols)} features using {method} method")
    return train_df, val_df, test_df, stats

In [ ]:
%%writefile apps/trainer/train/dataset.py
"""PyTorch dataset for sequences."""
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset
from loguru import logger

class SequenceDataset(Dataset):
    """Dataset for time series sequences."""

    def __init__(
        self,
        df: pd.DataFrame,
        feature_columns: list[str],
        sequence_length: int = 60,
        prediction_horizon: int = 15,
        prediction_type: str = "direction",
    ):
        self.df = df.reset_index(drop=True)
        self.feature_columns = feature_columns
        self.sequence_length = sequence_length
        self.prediction_horizon = prediction_horizon
        self.prediction_type = prediction_type

        # Precompute features and targets
        self.features = self.df[feature_columns].values.astype(np.float32)
        self._precompute_targets()

        # Valid indices (enough history + future)
        self.valid_indices = np.arange(
            sequence_length,
            len(self.df) - prediction_horizon
        )
        
        logger.info(
            f"Created dataset: {len(self.valid_indices)} sequences, "
            f"{len(feature_columns)} features, prediction_type={prediction_type}"
        )

    def _precompute_targets(self):
        """Precompute target values."""
        future_close = self.df["close"].shift(-self.prediction_horizon)
        current_close = self.df["close"]
        
        if self.prediction_type == "direction":
            self.targets = (future_close > current_close).astype(np.float32).values
        else:
            self.targets = ((future_close - current_close) / current_close).astype(np.float32).values

    def __len__(self) -> int:
        return len(self.valid_indices)

    def __getitem__(self, idx: int) -> tuple[torch.Tensor, torch.Tensor]:
        actual_idx = self.valid_indices[idx]
        
        # Get sequence
        seq_start = actual_idx - self.sequence_length
        seq_end = actual_idx
        sequence = self.features[seq_start:seq_end]
        
        target = self.targets[actual_idx]
        
        return torch.from_numpy(sequence), torch.tensor([target], dtype=torch.float32)

In [ ]:
%%writefile evaluate_gpu.py
"""GPU-accelerated model evaluation."""
import sys
from pathlib import Path
from typing import Dict

import numpy as np
import pandas as pd
import torch
from loguru import logger
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torch.utils.data import DataLoader

# Add modules to path
sys.path.insert(0, '/content')

from apps.trainer.models.lstm import LSTMModel
from apps.trainer.data_pipeline import load_features_from_parquet, create_walk_forward_splits
from apps.trainer.features import normalize_features, FEATURE_COLUMNS
from apps.trainer.train.dataset import SequenceDataset

def evaluate_model(
    model_path: str,
    symbol: str,
    device: str = "cuda",
    batch_size: int = 64,
) -> Dict:
    """Evaluate model on GPU."""
    logger.info(f"🚀 Evaluating {symbol} on {device.upper()}")
    
    # Load data
    df = load_features_from_parquet(symbol)
    train_df, val_df, test_df = create_walk_forward_splits(df)
    
    # Normalize
    train_df, val_df, test_df, stats = normalize_features(train_df, val_df, test_df)
    
    # Create dataset
    feature_cols = [c for c in FEATURE_COLUMNS if c in test_df.columns]
    test_dataset = SequenceDataset(
        test_df,
        feature_columns=feature_cols,
        sequence_length=60,
        prediction_horizon=15,
        prediction_type="direction",
    )
    
    test_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=2,
        pin_memory=True if device == "cuda" else False,
    )
    
    # Load model
    model = LSTMModel(
        input_size=len(feature_cols),
        hidden_size=128,
        num_layers=3,
        dropout=0.2,
        bidirectional=True,
    )
    
    checkpoint = torch.load(model_path, map_location=device)
    model.load_state_dict(checkpoint["model_state_dict"])
    model = model.to(device)
    model.eval()
    
    logger.info(f"Evaluating on {len(test_dataset):,} sequences...")
    
    # Evaluate
    all_preds = []
    all_targets = []
    all_probs = []
    
    with torch.no_grad():
        for batch_idx, (sequences, targets) in enumerate(test_loader):
            sequences = sequences.to(device)
            targets = targets.to(device)
            
            outputs = model(sequences)
            probs = outputs.squeeze().cpu().numpy()
            preds = (probs >= 0.5).astype(int)
            
            all_probs.extend(probs.tolist() if probs.ndim > 0 else [probs.item()])
            all_preds.extend(preds.tolist() if preds.ndim > 0 else [preds.item()])
            all_targets.extend(targets.squeeze().cpu().numpy().tolist())
            
            if batch_idx % 100 == 0:
                progress = (batch_idx + 1) * batch_size
                total = len(test_dataset)
                logger.info(f"  Progress: {progress:,}/{total:,} ({100*progress/total:.1f}%)")
    
    # Calculate metrics
    accuracy = accuracy_score(all_targets, all_preds)
    precision = precision_score(all_targets, all_preds, zero_division=0)
    recall = recall_score(all_targets, all_preds, zero_division=0)
    f1 = f1_score(all_targets, all_preds, zero_division=0)
    
    # Calibration error (simplified ECE)
    bins = np.linspace(0, 1, 11)
    bin_indices = np.digitize(all_probs, bins) - 1
    ece = 0.0
    for i in range(10):
        mask = bin_indices == i
        if mask.sum() > 0:
            bin_accuracy = np.array(all_targets)[mask].mean()
            bin_confidence = np.array(all_probs)[mask].mean()
            ece += mask.sum() / len(all_targets) * abs(bin_accuracy - bin_confidence)
    
    results = {
        "symbol": symbol,
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "calibration_error": ece,
        "num_samples": len(all_targets),
    }
    
    logger.info(f"✅ {symbol} Results:")
    logger.info(f"  Accuracy: {accuracy:.4f}")
    logger.info(f"  Precision: {precision:.4f}")
    logger.info(f"  Recall: {recall:.4f}")
    logger.info(f"  F1: {f1:.4f}")
    logger.info(f"  Calibration Error: {ece:.4f}")
    
    # Check promotion gates
    passes_accuracy = accuracy >= 0.68
    passes_calibration = ece <= 0.05
    
    logger.info(f"\n🎯 Promotion Gates:")
    logger.info(f"  Accuracy ≥ 68%: {'✅ PASS' if passes_accuracy else '❌ FAIL'} ({accuracy:.2%})")
    logger.info(f"  Calibration ≤ 5%: {'✅ PASS' if passes_calibration else '❌ FAIL'} ({ece:.2%})")
    logger.info(f"  Overall: {'✅ PROMOTION APPROVED' if (passes_accuracy and passes_calibration) else '❌ NOT READY'}")
    
    return results

if __name__ == "__main__":
    device = "cuda" if torch.cuda.is_available() else "cpu"
    logger.info(f"Using device: {device}")
    
    symbols = ["BTC-USD", "ETH-USD", "SOL-USD"]
    all_results = {}
    
    for symbol in symbols:
        model_path = f"models/new/lstm_{symbol.replace('-', '_')}_1m_7b5f0829.pt"
        logger.info(f"\n{'='*60}")
        logger.info(f"Evaluating {symbol}")
        logger.info(f"{'='*60}")
        
        results = evaluate_model(model_path, symbol, device=device)
        all_results[symbol] = results
    
    # Summary
    logger.info(f"\n\n{'='*60}")
    logger.info("📊 EVALUATION SUMMARY")
    logger.info(f"{'='*60}")
    
    results_df = pd.DataFrame(all_results).T
    print(results_df.to_string())
    
    # Save results
    results_df.to_csv("evaluation_results.csv")
    logger.info(f"\n✅ Results saved to evaluation_results.csv")

## Run Evaluation

This will evaluate all 3 models on GPU in parallel batches.

In [ ]:
!python evaluate_gpu.py

## Download Results

Download the evaluation results CSV file.

In [ ]:
from google.colab import files
files.download('evaluation_results.csv')